In [183]:
# import required libries
import os
import numpy as np
import spicy as sp
import pandas as pd

# define required path
raw_data = '../data/raw/'
curated_data = '../data/curated/'

In [184]:
suburb = pd.read_csv(curated_data + 'suburb.csv')
del suburb['Unnamed: 0']

In [185]:
# drop duplicate suburbs
suburb['Suburb'] = suburb['Suburb'].drop_duplicates()
suburb = suburb.dropna()

In [186]:
# read in dataset
properties = pd.read_csv(curated_data + 'properties.csv')

In [187]:
suburb_count = properties.groupby(['suburb'],as_index=False)['name'].count()
suburb_count.columns = ['suburb', 'count']

In [188]:
suburb_count['suburb'] = suburb_count['suburb'].str.upper()

In [189]:
suburb_count['suburb'][1]

'(CNR) TRARALGON'

In [190]:
suburb_count = suburb[['Suburb', 'SA2_code']].join(suburb_count.set_index('suburb'), on = 'Suburb').dropna().reset_index(drop=True)

In [191]:
sa2_count = suburb_count.groupby(['SA2_code'], as_index=False).sum()
sa2_count.columns = ['SA2_code', 'sum']

In [192]:
suburb_count = suburb_count.join(sa2_count.set_index('SA2_code'), on = 'SA2_code')

In [193]:
suburb_count

,Suburb,SA2_code,count,sum
0,MELBOURNE,206041122.0,482.0,482.0
1,EAST MELBOURNE,206041119.0,32.0,32.0
2,WEST MELBOURNE,206041127.0,43.0,43.0
3,SOUTHBANK,206051132.0,149.0,216.0
4,DOCKLANDS,206041118.0,111.0,111.0
...,...,...,...,...
639,DALYSTON,205031093.0,1.0,16.0
640,CAPE PATERSON,205031093.0,1.0,16.0
641,NORTH WONTHAGGI,205031093.0,1.0,16.0
642,WONTHAGGI,205031093.0,5.0,16.0


In [194]:
suburb_count['popu_ratio'] = suburb_count['count'] / suburb_count['sum']

In [195]:
suburb_count

,Suburb,SA2_code,count,sum,popu_ratio
0,MELBOURNE,206041122.0,482.0,482.0,1.000000
1,EAST MELBOURNE,206041119.0,32.0,32.0,1.000000
2,WEST MELBOURNE,206041127.0,43.0,43.0,1.000000
3,SOUTHBANK,206051132.0,149.0,216.0,0.689815
4,DOCKLANDS,206041118.0,111.0,111.0,1.000000
...,...,...,...,...,...
639,DALYSTON,205031093.0,1.0,16.0,0.062500
640,CAPE PATERSON,205031093.0,1.0,16.0,0.062500
641,NORTH WONTHAGGI,205031093.0,1.0,16.0,0.062500
642,WONTHAGGI,205031093.0,5.0,16.0,0.312500


In [196]:
popu = pd.read_csv(raw_data + 'population.csv')

In [197]:
required = ['SA2_code','2014', '2015', '2016', '2017', '2018', '2021']
popu = popu[required]

In [198]:
# Select data in Victoria
popu = popu.loc[(popu['SA2_code'].astype(int)>199999999) & (popu['SA2_code'].astype(int)<300000000)]

In [199]:
popu.to_csv(curated_data + 'population.csv')

In [200]:
suburb_popu = suburb_count.join(popu.set_index('SA2_code'), on = 'SA2_code')

In [201]:
years = ['2014', '2015', '2016', '2017', '2018', '2021']
suburb_popu[years] = suburb_popu[years].multiply(suburb_popu['popu_ratio'], axis="index").astype(int)

In [202]:
suburb_popu = suburb_popu.drop(['SA2_code', 'count', 'sum', 'popu_ratio'], axis=1)

In [203]:
suburb_popu.to_csv(curated_data + 'suburb_population.csv')

In [5]:
health = pd.read_csv(raw_data + 'health.csv')

In [6]:
health['Proportion'] = health['Proportion'].str.replace("*","")

/tmp/ipykernel_614/1757025872.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  health['Proportion'] = health['Proportion'].str.replace("*","")


In [7]:
health.to_csv(curated_data + 'health.csv')

NameError: name 'health_data' is not defined